In [211]:
import pandas as pd
import numpy as np

In [212]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
# Проведем слияния с основной базой базы поглощений

acq = pd.read_csv('/content/drive/MyDrive/Хакатон/datasets_to_merge_final/acquisitions.csv')

In [214]:
acq.sample()

,id,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,source_url,source_description,created_at,updated_at
660,661,796,c:15389,c:1004,NaN,0.0,USD,2008-12-15,http://uk.techcrunch.com/2008/12/05/tioti-sold...,TIOTI sold to Vizimo,2008-12-16 00:50:25,2008-12-16 00:50:25


In [215]:
acq.acquired_at = acq.acquired_at.fillna(acq.created_at)

In [216]:
first_acq = acq.groupby('acquiring_object_id')['acquired_at'].min().reset_index()
last_acq = acq.groupby('acquiring_object_id')['acquired_at'].max().reset_index()

In [217]:
acquiring_objects = pd.merge(first_acq, last_acq, how='outer', on='acquiring_object_id', suffixes=['_first', '_last'])

In [218]:
price_amount = acq.groupby('acquiring_object_id')['price_amount'].sum().reset_index()

In [219]:
acquiring_objects = acquiring_objects.merge(price_amount, how='outer', on='acquiring_object_id')

In [220]:
def agg_strings(lst):
    return ', '.join(list(filter(lambda x: isinstance(x, str), lst)))

terms = acq.groupby('acquiring_object_id').agg({'term_code': agg_strings}).reset_index()

In [221]:
acquiring_objects = acquiring_objects.merge(terms, how='outer', on='acquiring_object_id')

In [222]:
acquired_objects = acq.groupby('acquiring_object_id').agg({'acquired_object_id': agg_strings}).reset_index()

In [223]:
acquiring_objects = acquiring_objects.merge(acquired_objects, how='outer', on='acquiring_object_id')

In [224]:
acquiring_objects.sample()

,acquiring_object_id,acquired_at_first,acquired_at_last,price_amount,term_code,acquired_object_id
1954,c:26486,2008-06-19,2013-07-25,182500000.0,"cash, cash, cash","c:26487, c:27414, c:478, c:30586, c:3445, c:3525"


In [225]:
df = pd.read_pickle('/content/drive/MyDrive/Хакатон/t_y_obj_companies.pkl')

In [226]:
df.sample()

,id,entity_id,name,normalized_name,category_code,status,founded_at,closed_at,domain,homepage_url,...,seed_profile,y_combinator_year,city_2023,country_2023,Industries,Employees,homepage_url_2023,Company_values,Org_structure,Teams and qty
182273,c:56797,56797.0,Spinback,spinback,web,2.0,NaT,NaT,spinback.com,http://spinback.com,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [227]:
# Добавляем признаки acquired_at_first	acquired_at_last	price_amount	term_code	acquired_object_id

df = df.merge(acquiring_objects, how='left', left_on='id', right_on='acquiring_object_id')

In [228]:
df.sample()

,id,entity_id,name,normalized_name,category_code,status,founded_at,closed_at,domain,homepage_url,...,homepage_url_2023,Company_values,Org_structure,Teams and qty,acquiring_object_id,acquired_at_first,acquired_at_last,price_amount,term_code,acquired_object_id
221030,NaN,NaN,xCloud,xcloud,Mobile,1.0,2009-11-01,NaT,xcloud.cc,http://www.xcloud.cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
# Теперь обработаем со стороны купленного 

acquired_by = acq.groupby('acquired_object_id').agg({'acquiring_object_id': agg_strings}).reset_index()

In [230]:
acquired_at = acq.groupby('acquired_object_id')['acquired_at'].max().reset_index()

In [231]:
price_of_acq = acq.groupby('acquired_object_id')['price_amount'].sum().reset_index()

In [232]:
terms = acq.groupby('acquired_object_id').agg({'term_code': agg_strings}).reset_index()

In [233]:
acquired_by = acquired_by.merge(acquired_at, how='outer', on='acquired_object_id')

In [234]:
acquired_by = acquired_by.merge(price_of_acq, how='outer', on='acquired_object_id')

In [235]:
acquired_by = acquired_by.merge(terms, how='outer', on='acquired_object_id')

In [236]:
acquired_by = acquired_by.rename(columns={'acquired_object_id': 'id', 'acquiring_object_id': 'acquired_by',
       'price_amount': 'final_sell_price', 'term_code' : 'acquired_on_terms'})

In [237]:
acquired_by.sample()

,id,acquired_by,acquired_at,final_sell_price,acquired_on_terms
4199,c:277631,c:277632,2012-05-16,0.0,cash_and_stock


In [238]:
# Добавим признаки acquired_by	acquired_at	final_sell_price	acquired_on_terms

df = df.merge(acquired_by, how='left', on='id')

In [240]:
df.sample()

,id,entity_id,name,normalized_name,category_code,status,founded_at,closed_at,domain,homepage_url,...,acquiring_object_id,acquired_at_first,acquired_at_last,price_amount,term_code,acquired_object_id,acquired_by,acquired_at,final_sell_price,acquired_on_terms
219374,c:262760,262760.0,Wooroos,wooroos,ecommerce,1.0,2013-07-01,NaT,wooroos.com,http://www.wooroos.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
df.to_pickle('/content/drive/MyDrive/Хакатон/companies_code_freeze.pkl')